# Fire Truck Project Report

## Table of Contents

1. [Introduction](#introduction)
2. [Vision Module](#vision-module)
   - 2.1 [Overview](#overview)
   - 2.2 [Reading the Predefined Grid](#reading-the-predefined-grid)
   - 2.3 [Feature Extraction](#feature-extraction)
   - 2.4 [QR Code Identification](#qr-code-identification)
   - 2.5 [Objectives](#objectives)
      - 2.5.1 [Determining the Robot's Position](#determining-the-robots-position)
      - 2.5.2 [Recognizing Obstacles (In Red)](#recognizing-obstacles)
      - 2.5.3 [Identifying the Goal (In Blue)](#identifying-the-goal)
   - 2.6 [Pre-processing for Vision](#pre-processing)
   - 2.7 [Mapping and Localization](#mapping-and-localization)

3. [Optimal Path Computation](#optimal-path-computation)
   - 3.1 [Pathfinding Precision](#pathfinding-precision)
   - 3.2 [A* Algorithmic Mastery](#a-algorithmic-mastery)
   - 3.3 [Navgational Options](#navigational-options)
   - 3.4 [Path Reconstruction Elegance](#path-reconstruction-elegance)

4. [Motion Control Implementation](#motion-control-implementation)
   - 4.1 [Initialization and Configuration](#initialization-and-configuration)
   - 4.2 [Sensor Data Retrieval and Actuation](#sensor-data-retrieval-and-actuation)
   - 4.3 [Trajectory Following Mechanism](#trajectory-following-mechanism)

5. [Local Navigation Strategy](#local-navigation-strategy)
   - 5.1 [Assumptions](#assumptions)
   - 5.2 [Operational Assumptions](#operational-assumptions)
   - 5.3 [Obstacle Detection and Handling](#obstacle-detection-and-handling)
   - 5.4 [Maneuvering Strategies](#maneuvering-strategies)

6. [Kalman Filter in Localization](#kalman-filter-in-localization)
   - 6.1 [Adaptive Position Estimation](#adaptive-position-estimation)
   - 6.2 [Compensating for Uncertainties](#compensating-for-uncertainties)


## **1. Introduction<a name="introduction"></a>**

This project aims to integrate vision, path planning, local navigation, and filtering to guide a Thymio robot on a map towards a goal. We have developed two implementations: one guided by vision and another without relying on vision. The main goal is to create a robotic system with advanced features that is comparable to an autonomous fire rescue truck. 

In the initial setup, a webcam captures the surroundings of the experimental area. Real-time processing using traditional image techniques extracts essential map details like the robot's position, the map itself, fixed obstacles, and the destination. The A* algorithm then calculates the best route, relaying instructions to the global controller of the Thymio robot. This controller, in turn, directs the motors to follow the optimal path. If the Thymio detects an obstacle ahead through its proximity sensors, local navigation takes charge, guiding the robot to avoid collisions. Additionally, even if the Thymio is moved unexpectedly, it can still autonomously navigate its way to the intended destination.

Our project's story is based on a simulated scenario in which an autonomous fire rescue robot sets out on a mission to locate and rescue people trapped in a burning building, taking inspiration from the urgent and essential nature of firefighting scenarios. This hypothetical scenario highlights the necessity for robotic systems to manoeuvre through dangerous situations, assess the situation, and carry out precision rescue operations. It also matches the urgency and complexity of actual firefighting operations where speed and efficiency are keys for a successful salvation of the endangered victims.

### **Environment**

The operational canvas of our robot unfolds across a dynamic terrain represented by a grid-type map. Comprising 49 squares (7 rows and 7 columns), the map features white pathways and black barriers. ArUco markers strategically designate corners and the robot's position. The goal and the obstacles are respectively described by a blue square and a red square. 

</div style="width: 30%;">
    <img src="./map.jpeg" />
    <style="width: 100%;">
    <p style="text-align: center;">Figure 1: Map setup </p>
</div>

This environment facilitates the interplay of the vision module and the Thymio robot, allowing it to follow the global navigation path and dynamically adapt to obstacles using local navigation. A Kalman filter implementation further augments precision, incorporating equations for x and y coordinates, Thymio's angle, and motor velocities.

### **Structure**

In our project's organisational structure, a central "Command Centre" assumes a pivotal role as the hub for coordinating the cooperation of many modules, ensuring the seamless functioning of our self-contained fire rescue robot. This crucial module takes on the tasks of establishing the Thymio connection, carrying out careful camera calibration to maximise colour and edge detection, calculating the grid map, and drawing the route for navigation. This command centre is modular in operation and handles critical robot duties such as taking quick decisions regarding the presence and absence of an obstacle. The robot operates in two different modes: local navigation, which allows for real-time obstacle avoidance, and global navigation, which plots a route to the closest checkpoint. While maintaining knowledge of the position and orientation of the robot using either the Camera module or the Kalman filter. The diagram presented below illustrates the overarching structure of our project.

</div style="width: 30%;">
    <img src="./general_diagram.jpeg" />
    <style="width: 100%;">
    <p style="text-align: center;">Figure 2: General Diagram of the overall structure of the project</p>
</div>


The main variables : 

grid :
obs :
obs_grid :
targets :
goal :
width : 
height :
pos : 
angle : 
path : 
visitedNodes :
vector_to_checkpoint : 
### **Modules**

#### **Robot State Estimation with Extended Kalman Filter**

In our autonomous ambulance rescue robot project, accurate position and speed estimation are vital for effective navigation. We utilize an Extended Kalman Filter (EKF) implemented in the `KalmanFilter` class. The state space representation is tailored to our specific needs, considering a 4-dimensional state vector \([ \text{x}, \text{y}, \text{v}_x, \text{v}_y ]\), where \(\text{v}_x\) and \(\text{v}_y\) represent the linear velocities in the x and y directions, respectively.


##### **Coordinate System**

The coordinate system follows the convention shown below.

Here, \(x\) and \(y\) are in millimeters, and \(\text{{vx}}, \text{{vy}}\) are in millimeters per second.

##### **Discrete Model**

The discrete motion model for the Kalman filter is given by:

\[
\begin{align*}
\mathbf{z}_{k+1} &= f(\mathbf{z}_k) \\
\mathbf{z}^+ &= \begin{bmatrix} x + \frac{v_r + v_l}{2} \cos(\theta) T_s \\ y - \frac{v_r + v_l}{2} \sin(\theta) T_s \\ \text{{vx}} \\ \text{{vy}} \end{bmatrix} 
\end{align*}
\]

Here, \(k_s\) is the constant factor, and \(T_s\) is the time between two iterations. The covariance matrix \(\mathbf{Q}\) represents the motion model's uncertainty.

##### **Measurement Model**

The discrete measurement model is given by:

\[
\begin{align*}
\mathbf{z}_{k+1} &= h(\mathbf{z}_k) \\
\mathbf{z}^+ &= \begin{bmatrix} x_{\text{{measured}}} \\ y_{\text{{measured}}} \\ v_{r \text{{ measured}}} \\ v_{l \text{{ measured}}} \end{bmatrix} + \mathbf{v}
\end{align*}
\]

The measurement model incorporates data from the camera and the motor encoders, providing measured values for position and motor speeds. The covariance matrix \(\mathbf{R}\) captures the measurement noise.

##### **Extended Kalman Filter Algorithm**

The steps of the extended Kalman filter algorithm, tailored to our implementation, are as follows:

1. **Prediction Step:**
In this step, the Kalman filter predicts the next state of the system based on the previous state estimate and the system dynamics, incorporating the process model. The predicted state and covariance represent the system's expected behavior, considering motion and process uncertainties :

   \begin{align*}
   \hat{\mathbf{x}}_{k \mid k-1} &= f(\hat{\mathbf{x}}_{k-1 \mid k-1}, \mathbf{u}_k) \\
   \mathbf{P}_{k \mid k-1} &= \mathbf{F}_k \mathbf{P}_{k-1 \mid k-1} \mathbf{F}_k^T + \mathbf{Q}_k
   \end{align*}

2. **Measurement Residual:**
The measurement residual calculates the difference between the predicted measurement (obtained from the predicted state) and the actual measurement. It represents the discrepancy or error between the expected and observed values, capturing the system's deviation from the predicted state : 

   \begin{align*}
   \tilde{\mathbf{y}}_k &= \mathbf{z}_k - h(\hat{\mathbf{x}}_{k \mid k-1})
   \end{align*}

3. **Measurement Update:**
In the measurement update step, the Kalman filter refines its state estimate by incorporating the measured values. The Kalman gain is calculated to give more weight to reliable measurements and less weight to uncertain ones. The updated state estimate and covariance reflect a more accurate representation of the system's true state, considering both prediction and measurement inform : 

   \begin{align*}
   \mathbf{S}_k &= \mathbf{H}_k \mathbf{P}_{k \mid k-1} \mathbf{H}_k^T + \mathbf{R}_k \\
   \mathbf{K}_k &= \mathbf{P}_{k \mid k-1} \mathbf{H}_k^T (\mathbf{S}_k)^{-1} \\
   \hat{\mathbf{x}}_{k \mid k} &= \hat{\mathbf{x}}_{k \mid k-1} + \mathbf{K}_k \tilde{\mathbf{y}}_k \\
   \mathbf{P}_{k \mid k} &= (\mathbf{I} - \mathbf{K}_k \mathbf{H}_k) \mathbf{P}_{k \mid k-1}
   \end{align*}


##### **Estimation of Ts (Time difference)**

The accurate estimation of the time between iterations (\(T_s\)) is essential for the dynamic performance of the Kalman filter, directly influencing temporal alignment, motion prediction, and covariance matrix updates. A precise \(T_s\) ensures that predictions align with the system's temporal dynamics, maintaining the filter's adaptability to dynamic changes. Inaccuracies in \(T_s\) can disrupt temporal alignment, affecting the filter's ability to provide accurate state estimates and impacting control actions.

##### **Estimation of Noise Matrices**

For the motion model:<br />
$$
Q=\begin{bmatrix}
0.04 & 0 & 0 & 0\\
0 & 0.04 & 0 & 0\\
0 & 0 & 6 & 0\\
0 & 0 & 0 & 6
\end{bmatrix}
$$

We derived the values of 0.04 and the angle through empirical estimation, without a specific deterministic basis. Regarding the speed, empirical data obtained during the eighth week informed our estimation process, wherein half of the variance was attributed to the inherent motion dynamics, while the remaining half originated from measurement uncertainties. <br>

For the measurement model:<br />
$$
R=\begin{bmatrix}
0.25 & 0 & 0 & 0\\
0 & 0.25 & 0 & 0\\
0 & 0 &6 & 0\\
0 & 0 & 0 & 6
\end{bmatrix}
$$

These values are estimated, with half of the variance attributed to motion and the other half to measurement.


## 2. Vision Module<a name="vision-module"></a>

### 2.1 Overview<a name="overview"></a>

The vision module serves as the visual intelligence of the Thymio robot, forming a critical component within the broader project framework. Its primary purpose revolves around interpreting the surrounding environment through a predefined grid, feature extraction, and the recognition of specific visual cues, such as QR codes. The overarching objectives of this module are multi-faceted and include determining the robot's precise position, identifying obstacles marked in red, pinpointing the goal represented in blue, and detecting a secondary goal highlighted in green.

```python
import numpy as np
import cv2 as cv
from math import floor
from enum import Enum
from vision.utils import print_error
from vision.constants import grid_height_cells, grid_width_cells
```

```python
    class VisionObjectsTypes(Enum):
        GENERAL = 0
        OBSTACLE = 1
        GOAL = 2
        TARGET = 3
```

The vision module starts by importing necessary libraries for image processing, numerical operations, and custom utility functions and constants needed for this project. The enumeration class presented above defines different types of vision objects: general, obstacle, goal, and target.

### 2.2 Reading the Predefined Grid<a name="reading-the-predefined-grid"></a>

The core functionality of the vision module lies in its ability to comprehend a predefined grid. This involves processing visual data to interpret the grid's layout, which serves as the navigational reference for the Thymio robot. By parsing the grid, the vision module equips the robot with essential spatial awareness, enabling it to make informed decisions about its movements and thus allowing more complex control of the thymio such as the A* global path computation or the P controller for the robot's movements.

### 2.3 Feature Extraction<a name="feature-extraction"></a>

To navigate effectively, the vision module engages in feature extraction, isolating distinctive elements within the visual input. These features are in general the edges, corners and the unique patterns of the QR codes that aid in recognizing the environment. By extracting these features, we gain a more detailed understanding the surroundings and utilising this to precise localization and navigation.

### 2.4 QR Code Identification<a name="qr-code-identification"></a>

The vision module is designed to identify QR codes, leveraging their encoded information for various purposes within the project such as conveying data regarding the Thymio's position, orientation and the Grid's corners.

### 2.5 Objectives<a name="objectives"></a>

#### 2.5.1 Determining the Robot's Position<a name="determining-the-robots-position"></a>

A fundamental objective of the vision module is to accurately determine the Thymio robot's position within its environment. This involves integrating information from the predefined grid, extracted features, and QR codes to establish a real-time understanding of the robot's spatial coordinates, speed and orienation.

#### 2.5.2 Recognizing Obstacles (In Red)<a name="recognizing-obstacles"></a>

The vision module is tasked with identifying obstacles marked in red. This capability enables the Thymio robot to proactively detect and respond to potential hindrances in its path, facilitating efficient and safe navigation. Using the camera to locate the predefined obstacles in red allows the robot to only detect any new detected obstacle in it's path.

#### 2.5.3 Identifying the Goal (In Blue)<a name="identifying-the-goal"></a>

Recognizing the goal, indicated in blue, is important for the robot's navigation strategy. The vision module's ability to identify the goal allows the Thymio robot to plan and execute optimal paths using algorithms (in our case the __A*__), steering towards final destinations.

In essence, the vision module serves as the "eyes" of the Thymio robot, enabling it to perceive, interpret, and respond intelligently to the visual cues present in its environment. The successful execution of these vision objectives forms the foundation for the robot's overall navigation and mission accomplishment.

### 2.6 Pre-processing for Vision<a name="pre-processing"></a>

The original frames captured by the camera undergo pre-processing to address issues like chromatic aberration, noise, and unstandardization caused by changes in camera position and shooting angle. The map region is extracted using markers, and perspective projection is removed with a projective transform for robustness. The image is converted to grayscale, filtered for noise with a Gaussian filter, and thresholded for binary mapping. Future improvements may include exploring different filters or thresholding methods to enhance performance under varied conditions.

### 2.7 Mapping and Localization<a name="mapping-and-localization"></a>

After pre-processing, the image is cropped into grids, and the vision module identifies walls and goals using ArUco markers. The robot's position and rotation gained by the vision module are used in conjunction with motor speed estimates for a more precise state estimation. This information is then utilized in control and navigation modules, providing a comprehensive understanding of the robot's real-time state.

Certainly! Here is the modified section, including the provided assumptions, integrated into the local navigation part:


## 3. Optimal Path Computation<a name="optimal-path-computation"></a>

### 3.1 Pathfinding Precision<a name="pathfinding-precision"></a>

This section will be focusing on global navigation, unraveling the choices of the Thymio's optimal path computation facilitated by the __A* algorithm__. Beyond mere movement, this algorithmic approach aspires to chart the most efficient course for the robot, navigating through a predefined grid with discernment minimising the cost needed to achieve the desired result.

### 3.2 A* Algorithmic Mastery<a name="a-algorithmic-mastery"></a>


#### 3.3 Navigational Options<a name="navigational-options"></a>
Two distinct sets of movement options are already provided for the Thymio's trajectory within the A* algorithm. Both 4-connectivity and 8-connectivity movements are optimally computed. After careful consideration, we have chosen to use the 4N connectivity for our project. This decision is informed by the nature of the environment, which tries to mirrors a realistic scenario with streets and buildings where diagonal movements are restricted. The adoption of 4N connectivity aligns with the practical constraints of the project, ensuring computational efficiency while navigating through the constrained layout of the simulated environment.


#### 3.4 Path Reconstruction Elegance<a name="path-reconstruction-elegance"></a>

The reconstruction of the path is important in how we make the Thymio move. This way of doing things in the __A* algorithm's__ code ensures that we can efficiently figure out the best path for the Thymio to follow. It's like setting up a smooth roadmap that helps the Thymio navigate carefully and reach its destination.

## 4. Motion Control Implementation Overview<a name="motion-control-implementation-overview"></a>

### 4.1 Initialization and Configuration<a name="initialization-and-configuration"></a>

The motion control process initiates with the instantiation of the `Control` class, where parameters for motion control, including the proportional (kp), integral (ki), and derivative (kd) gains, as well as the robot's wheel base width, are defined. These parameters establish the foundation for subsequent motion control strategies. Additionally, the class contains methods for sensor data retrieval and actuation, contributing to the overall adaptability of the robot's motion.

### 4.2 Sensor Data Retrieval and Actuation<a name="sensor-data-retrieval-and-actuation"></a>

The `Control` class features a method named `get_sensors` responsible for retrieving sensor data such as motor speeds and horizontal proximity values. This method provides essential information for making informed decisions during the motion control process and also for the local avoidance tasks. The `set_motors_PID` method is implemented to adjust the left and right motor speeds based on PID control, enhancing the adaptability and responsiveness of the robot to environmental changes and thus limiting the angular error allowed for the robot.

### 4.3 PID Controller Implementation<a name="pid-controller-implementation"></a>

A PID controller is implemented within the `pid_controller` function, utilizing proportional, integral, and derivative terms to dynamically adjust motor speeds. The control gains (kp, ki, kd) and the robot's width play a crucial role in determining the correction applied to the motors. The function also includes a mechanism to handle aggressive corrections for significant angle deviations, ensuring effective turning when needed. The `normalize_angle` function is utilized to maintain the angle within the range of -180 to 180 degrees.

**Table of Constants for Motion Control:**

| Constant        | Value/Description                          |
|-----------------|--------------------------------------------|
| kp              | 2.0                                        |
| ki              | 0.04                                       |
| kd              | 0.01                                       |
| robot_width     | 0.1                                        |

This set of constants is meticulously chosen to balance the responsiveness and stability of the robot's motion control, ensuring effective navigation in various scenarios. The proportional, integral, and derivative gains are fine-tuned to achieve optimal performance, while the robot's width is considered for accurate motion calculations.

## 5. Local Navigation Strategy<a name="local-navigation-strategy"></a>

</div style="width: 30%;">
    <img src="./Local_nav_model.jpeg" />
    <style="width: 100%;">
    <p style="text-align: center;">Figure 3: Local Navigation System Operation Diagram</p>
</div>

The Local Navigation module is designed to respond dynamically to obstacles detected by the Thymio robot's proximity sensors. The goal is to trigger Local Navigation when an obstacle is detected, ensuring the robot can navigate around it effectively. Proximity sensors' captured values trigger Local Navigation when above a desired threshold distance.

#### 5.1 Assumptions<a name="local-assumptions"></a>

In our environment, some obstacles are strategically placed along the robot's path. The Local Navigation module operates under the following assumptions:

1. **Obstacle Blocking Assumption:**
   - The obstacles are completely blocking the robot's path. This choice aligns with our decision to perform Global Navigation only once at the program's beginning. This assumption allows Local Navigation to focus on avoiding specific obstacles without being constrained by the assumptions made during Global Navigation and thus releaving us from the necessity of recomputing the global path

2. **Light Reflective Obstacle Assumption:**
   - The obstacles are assumed to be light-reflective as the proximity sensors utilize infrared light. The sensors consist of a transmitter and a receiver, with the light reflecting off the obstacle. This assumption rules out obstacles that diffract light extensively, such as glass, ensuring reliable detection.

3. **Simple Convex Geometrical Section Assumption:**
   - Obstacles are assumed to have a simple convex geometric shape including triangles, rectangles and squares. This assumption facilitates the robot's ability to navigate around obstacles by following specific scenarios preimplemented. It ensures that the robot can circumvent obstacles without encountering complex concave shapes that might hinder navigation.

#### 5.2 Operational Assumptions<a name="local-operational-assumptions"></a>

Given these assumptions, the Local Navigation module specifically assumes that obstacles are square-shaped. When an obstacle is detected, the entire square is considered impassable, and the robot plans its path accordingly. The assumption further specifies that the obstacle is positioned centrally within the square.

This operational approach enables the robot to navigate around square obstacles effectively. However, it is important to note that the Local Navigation module does not require adjustements if different obstacle shapes are introduced due to the designed library of local paths to follow. The successful execution of Local Navigation relies on the accurate adherence to these assumptions for obstacle characteristics and the geometric layout of the environment.

### 5.3 Obstacle Detection and Handling<a name="obstacle-detection-and-handling"></a>


**Obstacle Identification:** 
The robot retrieves and scales data from the proximity sensor to identify new obstacles in its path that may not have been captured by the camera. The `local_nav()` function checks if the proximity sensor in front of the robot, denoted as `x[2]`, does not detect a new obstacle. Upon the threshold of 20 being exceeded by the value of `x[2]`, the system switches to the obstacle-handling state, triggering the `obstacle_function()` when the boolean variable `obstacle`is set to true.

**Obstacle Handling:**
 When an obstacle is encountered, the robot stops (`forward_robot(0)`) and assesses the position relative to the global path. Three scenarios are considered:
  - If the global path is behind the obstacle, the robot maneuvers to reach the path (`state = obstacle_behind`).
  - If the target is to the left of the obstacle, the robot executes a specific maneuver (`state = obstacle_left`).
  - If the target is to the right of the obstacle, the robot follows a different maneuver (`state = obstacle_right`). 
To determine the scenario, we conduct a comparison between the robot's position in front of the obstacle (`position[0]`)]) and the subsequent position in the global path (`path[0][0]`), taking into consideration the abscissae of the positions.


### 5.4 Maneuvering Strategies<a name="maneuvering-strategies"></a>

Depending on the selected senario, we will apply different maneuvering strategies. When an obstacle is encountered, the robot dynamically decides whether to turn left or right based on its initial target position relative to the obstacle. This decision influences the choice of one of the following functions. This approach enables the robot to adapt its maneuvering strategy, ensuring efficient obstacle avoidance and alignment with the next best location of the global path :

- **Maneuvering Behind Obstacle (`obstacle_behind`):** 
    ```python
        handle_obstacle_behind(path, special_cases)
    ```
- **Maneuvering Left of Obstacle (`obstacle_left`):**
    ```python
        handle_obstacle_left(position, obstacle_position)
    ```
- **Maneuvering Right of Obstacle (`obstacle_right`):**
    ```python
        handle_obstacle_right(position, obstacle_position)
    ```
When the target is situated behind an obstacle, two scenarios merit consideration:

- If the target is behind the obstacle, and the subsequent step in the global path does not align with the predefined special cases, the robot straightforwardly moves to the position behind the obstacle.
- If the target is behind the obstacle, and the next step in the global path corresponds to either of the two special cases previously defined (refer to the sketch above), it is more optimal for the robot to navigate directly to the second position in the global path, precisely on the special cases.

Finally, once the obstacle is navigated, we enter in `state = orientation`. The robot assesses the next step in the global path to adjust its orientation with the next step in the path.To decide if it has to turn or not we just compare the abscisse of the robot and the one of the next position in the path.

- **Next Step (`next_step`):** 
    ```python
       handle_target_state(path, k)
    ```

## 6. Kalman Filter in Localization<a name="kalman-filter-in-localization"></a>

### 6.1 Adaptive Position Estimation<a name="adaptive-position-estimation"></a>

The Kalman Filter is a pivotal component in our localization strategy, adapting to changing conditions and continuously refining its estimates based on motor speed inputs. We use the kalman mainly in the case of camera obscurity or perturbation to send the estimated position and angle of the robot to the control functions in real time. The Kalman is structures as follows : first the functoin called **compute_x_y_speed(self, left_motor_speed, right_motor_speed)** is called taking as inputs the left and right motor speeds, it then computes the speeds in term of linear speed (v_r - v_l)/2 , the angular speed, the the vectorial speeds [v_x and v_y] and finally uses the difference between the 2 wheels' speed to compute the angular variation $\Delta\theta$ to estimate the actual orienation and angle of the our Robot.

In our state space representation, we chose not to include the angle (\(\theta\)) as a primary state variable to enhance computational efficiency. Instead, we compute and estimate \(\theta\) separately using the motor inputs from sensor values. This modular approach streamlines computations, reduces complexity, and aligns with the specific requirements of our application. In addition, the speeds returned from the functions are then being passed through our extended Kalman Filter to ultimately conclude our estimation for the position which is the most important element for which we have the estimator. In addition to our global choices, we have decided to facilitate the readability of the filtering task and merge both steps explained above for this task : the **prediction** and the **update** in a single predict function that outputs the expected status of the system and the updated matrix **P** as well.

#### Constants used for the filtering 

| Constant                   | Value or Description            | Justification                                                                |
|-----------------------------|--------------------------------|------------------------------------------------------------------------------|
| `dt`                        | 0.08                                          | Time step for the state transition matrix by default untill the comuted time is passed to the function                    |
| `speed_conv_factor`         | 0.35                                           |Conversion factor for motor speed to linear speed computed by observing the robot for a distance of 12cm with a speed ot 80 and computing the \(\delta t\) thus extracting the factor             |
| `wheel_width`               | 100 (distance between the two wheels in mm)    | Distance between the two wheels of the robot used to compute the angular speed of the robot             |


```python
class KalmanFilter(object):
    def __init__(self, dim_x=4, dim_z=2, dt=0.1):
        # Initialization code

    def compute_x_y_speed(self, left_motor_speed, right_motor_speed):
        # Compute linear speed and angular speed
        return speed, new_angle
    
    def predict(self, left_motor_speed, right_motor_speed, orientation, x_est_prev, P_est_prev):
        # Measurement prediction covariance
        S = np.dot(self.H, np.dot(self.P_est, self.H.T)) + self.R

        # Kalman gain
        K = np.dot(self.P_est, np.dot(self.H.T, np.linalg.inv(S)))

        # A posteriori estimate
        self.x_est = self.x_est + np.dot(K, i)
        self.P_est = self.P_est - np.dot(K, np.dot(self.H, self.P_est))

        # Extracting relevant information for the output...
        return estimated_position, estimated_speed, angle, self.x_est, self.P_est
```

1. **`__init__(self, dim_x=4, dim_z=2, dt=0.1):`**
   - **Purpose:** Initializes the Kalman filter object with specified dimensions and time parameters.

2. **`compute_x_y_speed(self, left_motor_speed, right_motor_speed):`**
   - **Purpose:** Computes the linear speed (in x and y directions) and angular speed of the robot based on left and right motor speeds.
   - **Returns:**
     - `speed`: Linear speed of the robot.
     - `new_angle`: The robot's new estimated angle.

3. **`predict(self, left_motor_speed, right_motor_speed, orientation, x_est_prev, P_est_prev):`**
   - **Purpose:** Predicts the next state estimate using the Kalman filter based on motor speeds, orientation, and previous estimates.
   - **Returns:**
     - `estimated_position`: Estimated position of the robot.
     - `estimated_speed`: Estimated linear speed of the robot.
     - `angle`: Estimated angular speed of the robot.
     - `self.x_est`: Updated state estimate.
     - `self.P_est`: Updated state covariance estimate.

### 6.2 Compensating for Uncertainties<a name="compensating-for-uncertainties"></a>

To enhance the robustness of our localization system, we explicitly consider uncertainties in motor speed measurements and orientation through the process uncertainty matrix (`Q`) and measurement uncertainty matrix (`R`). The values computed in the code are either measured experimentally or extracted from the Practical session number 8 of the course of Mobie Robotics with Professor Francesco Mondada.

### **7 Conclusion**

We methodically created and included key components in this robotics project to imitate the operations of an autonomous fire rescue robot. The robot can determine its position, identify obstacles, and find goals thanks to the Vision Module's interpretation of a specified grid, feature extraction, and QR code recognition. The A* algorithm drives the Optimal Path Computation, which accurately determines the best path through a grid for the robot's precision navigation. The robot's movements are dynamically adjusted by the Motion Control Implementation, and obstructions detected by proximity sensors are skillfully responded to by the Local Navigation Strategy. In localization, the Kalman Filter improves position estimations by accounting for uncertainties. This study shows how an autonomous fire rescue robot can successfully navigate complex areas, avoid obstacles, and accomplish mission objectives.

### **Main Loop**


